# Importing Data

In [ ]:
import pandas as pd
import numpy as np
import itertools
from itertools import combinations
from datetime import datetime, timedelta
import time
import mysql.connector
from mysql.connector import Error

## Setting sample Battery scores
- Fake scores are used for now as real scores are not available at time of code's creation
- Scores here do not necessarily be of any amount, but I chose 3 batteries (Batt2, Batt3, Batt5) to = 1.0 (100% score) for testing purposes

In [ ]:
# Set the random seed for reproducibility
np.random.seed(80)

# Define the columns
columns = ['ID', 'BESS1_score', 'BESS2_score', 'BESS3_score', 'BESS4_score', 'BESS5_score']

# Define the number of rows
num_rows = 10 # Not necessary to be 10, pick any number you want

# Generate the random values for the battery columns
data = np.random.uniform(low=0.0, high=1.0, size=(num_rows, len(columns) - 1))

# Round the values to the nearest 2 decimal places
data = np.round(data, 2)

# Create the DataFrame
scoreTable = pd.DataFrame(data, columns=columns[1:])

scoreTable.iloc[9, 1] = 1.0  
scoreTable.iloc[9, 2] = 1.0 
scoreTable.iloc[9, 4] = 1.0 
print(scoreTable)

![Battery Scores](https://github.com/roof002/Dynamic-Battery-Choosing-Algorithm/blob/8047b2829b15978c14bb3096b0bb07335c3197a7/Images/BatteryScores.png)

# Get Functions

In [ ]:
def get_sum_sites(siteData):
    sites_values = siteData[['Site1', 'Site2', 'Site3', 'Site4', 'Site5']]
    return sites_values.sum()

In [ ]:
def get_sites_ind(siteData):
    site1 = siteData['Site1']
    site2 = siteData['Site2']
    site3 = siteData['Site3']
    site4 = siteData['Site4']
    site5 = siteData['Site5']
    
    # Return the values as a tuple
    return site1, site2, site3, site4, site5

In [ ]:
def get_discharge_limits():
    discharge_per_batt = {
            "Batt1": (450, 500),
            "Batt2": (350, 400),
            "Batt3": (200, 250),
            "Batt4": (200, 250),
            "Batt5": (150, 200)
        }
    return discharge_per_batt

In [ ]:
def get_new_site_row(connection):
    try:
        cursor = connection.cursor(dictionary=True)
        
        # Query to get the latest record from SitesPowerNow
        query_sites = """
        SELECT * FROM SitesPowerNow
        ORDER BY Date DESC
        LIMIT 1
        """
        cursor.execute(query_sites)
        latest_site_record = cursor.fetchone()

        if latest_site_record:
            df_latest_site_record = pd.DataFrame([latest_site_record])
            return df_latest_site_record
        else:
            return pd.DataFrame()

    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return pd.DataFrame()
    finally:
        if cursor:
            cursor.close()

In [ ]:
def get_new_batt_row(connection):
    try:
        cursor = connection.cursor(dictionary=True)
        
        # Query to get the latest record from BatteriesTable
        query_batt = """
        SELECT * FROM BatteriesTable
        ORDER BY ReadTime DESC
        LIMIT 1
        """
        cursor.execute(query_batt)
        latest_batt_record = cursor.fetchone()

        if latest_batt_record:
            df_latest_batt_record = pd.DataFrame([latest_batt_record])
            return df_latest_batt_record
        else:
            return pd.DataFrame()

    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return pd.DataFrame()
    finally:
        if cursor:
            cursor.close()

In [ ]:
def check_discharge_rates(siteData):
    # This is an example of how discharge limits might be defined
    discharge_limits = get_discharge_limits()

    # Adjust limits based on actual site values
    adjusted_limits = {}
    for i, (batt, (min_rate, max_rate)) in enumerate(discharge_limits.items()):
        site_value = siteData[f'Site{i+1}']
        if isinstance(site_value, pd.Series):
            site_value = site_value.values[0]  # Ensure it's a scalar value
        if site_value < min_rate:
            continue  # Exclude the battery if the site value is less than min_rate
        elif site_value > max_rate:
            adjusted_limits[batt] = (min_rate, max_rate)  # Keep original rate if site value is greater than max_rate
        else:
            adjusted_limits[batt] = (min_rate, site_value)  # Adjust max_rate to site value if within range

    return adjusted_limits


###### OUTPUT LOOK LIKE THIS ######
# Adjusted Limits: {'Batt1': (450, 480), 'Batt2': (350, 360), 'Batt3': (200, 220), 'Batt4': (200, 210), 'Batt5': (150, 180)}

In [ ]:
def get_score_for_batt(batt, scoreTable, rowIndex):
    batt_scores = {
        "Batt1": scoreTable.iloc[rowIndex]['BESS1_score'],
        "Batt2": scoreTable.iloc[rowIndex]['BESS2_score'],
        "Batt3": scoreTable.iloc[rowIndex]['BESS3_score'],
        "Batt4": scoreTable.iloc[rowIndex]['BESS4_score'],
        "Batt5": scoreTable.iloc[rowIndex]['BESS5_score']
    }
    return batt_scores[batt]

# MySQL Get Functions

In [ ]:
connection = mysql.connector.connect(
    host='Hostname',
    port='Port',
    user='Username',
    password='Password',
    database='Database',
    buffered=True,
    connection_timeout = 30
    )    

connection.autocommit = True

In [ ]:
def get_dfSites_from_mysql(connection):
    try:
        cursor = connection.cursor(buffered=True)
        # Execute the query to fetch and lock data
        cursor.execute("SELECT SQL_NO_CACHE Date, Site1, Site2, Site3, Site4, Site5 FROM SitesPowerNow")
        dfSites = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        dfSites = pd.DataFrame()
    
    finally:
        if 'cursor' in locals():
            cursor.close()
    
    return dfSites


In [ ]:
def get_dfActNot_from_mysql(connection):
    try:
        cursor = connection.cursor(buffered=True)
        # Execute the queries and fetch data
        cursor.execute("SELECT SQL_NO_CACHE ReceivedTime, TotalLoad, StartTime, EndTime FROM ActivationNotice")
        dfActNot = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
   
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        dfActNot = pd.DataFrame()
    
    finally:
        if 'cursor' in locals():
            cursor.close()
    
    return dfActNot

In [ ]:
def get_dfBatt_from_mysql(connection):
    try:
        cursor = connection.cursor(buffered=True)
        # Execute the queries and fetch data
        cursor.execute("SELECT SQL_NO_CACHE ReadTime, BESS1_Status, BESS1_Capacity, BESS2_Status, BESS2_Capacity, BESS3_Status, BESS3_Capacity, BESS4_Status, BESS4_Capacity, BESS5_Status, BESS5_Capacity FROM BatteriesTable")
        dfBatt = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
        
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        dfBatt = pd.DataFrame()
    
    finally:
        if 'cursor' in locals():
            cursor.close()
    
    return dfBatt

In [ ]:
def execute_query(connection, query, data):
    try:
        cursor = connection.cursor(dictionary=True)
        cursor.execute(query, data)
        connection.commit()
        print("Query executed successfully")
    except mysql.connector.Error as e:
        print(f"The error '{e}' occurred")
    finally:
        if 'cursor' in locals():
            cursor.close()

In [ ]:
def reconnect():
    try:
        connection = mysql.connector.connect(
        host='Hostname',
        port='Port',
        user='Username',
        password='Password',
        database='Database',
        buffered=True  
        )    
        if connection.is_connected():
            print("Successfully reconnected to the database")
            return connection
    except mysql.connector.Error as err:
        print(f"Error: Could not connect to the database: {err}")
    return None

In [ ]:
def get_specific_site_row(connection, timestamp):
    try:
        cursor = connection.cursor(dictionary=True)
        
        # Query to get the record from SitesPowerNow at the specified timestamp
        query = """
        SELECT * FROM SitesPowerNow
        WHERE Date = %s
        LIMIT 1
        """
        cursor.execute(query, (timestamp,))
        specific_site_record = cursor.fetchone()

        if specific_site_record:
            return pd.Series(specific_site_record)
        else:
            # If no exact match is found, fetch the nearest previous record
            query_nearest = """
            SELECT * FROM SitesPowerNow
            WHERE Date < %s
            ORDER BY Date DESC
            LIMIT 1
            """
            cursor.execute(query_nearest, (timestamp,))
            nearest_site_record = cursor.fetchone()
            return pd.Series(nearest_site_record) if nearest_site_record else pd.Series()

    except Error as e:
        print(f"Error: {e}")
        return pd.Series()
    finally:
        if cursor:
            cursor.close()

In [ ]:
def get_specific_batt_row(connection, target_time):
    try:
        cursor = connection.cursor(dictionary=True)
        
        # Query to get the specific record from BatteriesTable at the given time
        query_batt = """
        SELECT * FROM BatteriesTable
        WHERE ReadTime = %s
        """
        cursor.execute(query_batt, (target_time,))
        specific_batt_record = cursor.fetchone()

        if not specific_batt_record:
            # If no exact match is found, get the closest record before the target time
            query_batt = """
            SELECT * FROM BatteriesTable
            WHERE ReadTime < %s
            ORDER BY ReadTime DESC
            LIMIT 1
            """
            cursor.execute(query_batt, (target_time,))
            specific_batt_record = cursor.fetchone()

        return pd.Series(specific_batt_record) if specific_batt_record else pd.Series()

    except Error as e:
        print(f"Error: {e}")
        return pd.Series()
    finally:
        if cursor:
            cursor.close()

## Specific queries required

In [ ]:
insert_query = """
INSERT INTO SitesCommand (ReceiveTime, Site1_CnD, Site1_Amt, Site2_CnD, Site2_Amt, Site3_CnD, Site3_Amt, Site4_CnD, Site4_Amt, Site5_CnD, Site5_Amt)
VALUES (%(ReceiveTime)s, %(Site1_CnD)s, %(Site1_Amt)s, %(Site2_CnD)s, %(Site2_Amt)s, %(Site3_CnD)s, %(Site3_Amt)s, %(Site4_CnD)s, %(Site4_Amt)s, %(Site5_CnD)s, %(Site5_Amt)s)
"""

In [ ]:
end_query = """
INSERT INTO SitesCommand (ReceiveTime, Site1_CnD, Site1_Amt, Site2_CnD, Site2_Amt, Site3_CnD, Site3_Amt, Site4_CnD, Site4_Amt, Site5_CnD, Site5_Amt)
VALUES (%(ReceiveTime)s, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
"""

# Fractional Knapsack

In [ ]:
def calculate_combination_scores(all_combinations, totalLoad):
    scored_combinations = []

    for combo, total_value, initial_score in all_combinations:
        total_score = initial_score
        
        # Apply penalty after calculating the initial total score
        if total_value > totalLoad:  # lower penalty
            penalty = 0.001 * (total_value - totalLoad) / totalLoad
            total_score -= penalty
            print(f"Penalty applied (lower): {penalty} for combo: {combo}")
        elif total_value < totalLoad:  # higher penalty
            penalty = 0.1 * (totalLoad - total_value) / totalLoad
            total_score -= penalty
            print(f"Penalty applied (higher): {penalty} for combo: {combo}")
        
        scored_combinations.append((combo, total_value, total_score))
        print(f"Scored combination: {combo} with total score: {total_score}")

    # Remove duplicate combinations using frozenset for uniqueness
    unique_combinations = list({
        frozenset((k, frozenset(v.items())) for k, v in combo.items()): (combo, value, score)
        for combo, value, score in scored_combinations
    }.values())

    return unique_combinations



# [
#     (
#         {
#             'Batt1': {'value': 300, 'score': 0.9, 'min_rate': 450, 'max_rate': 500, 'adjusted_total': 675.0},
#             'Batt3': {'value': 50, 'score': 0.8, 'min_rate': 200, 'max_rate': 241.604, 'adjusted_total': 300.0}
#         },
#         350,
#         0.72  # 0.9 * 0.8
#     ),
#     (
#         {
#             'Batt2': {'value': 250, 'score': 0.7, 'min_rate': 350, 'max_rate': 400, 'adjusted_total': 525.0},
#             'Batt4': {'value': 100, 'score': 0.9, 'min_rate': 200, 'max_rate': 250, 'adjusted_total': 300.0}
#         },
#         350,
#         0.63  # 0.7 * 0.9
#     )
# ]

In [ ]:
def fractional_knapsack(totalLoad, siteData, scoreTable, rowIndex, connection, target_time, tolerance=10.0):

    # Get adjusted discharge limits based on the site values
    adjusted_discharge_limits = check_discharge_rates(siteData)
    print(f"Adjusted discharge limits: {adjusted_discharge_limits}")

    # Get battery capacities from the specific time row in the database
    batt_row = get_specific_batt_row(connection, target_time)
    if batt_row.empty:
        print(f"No battery data found for the specified time: {target_time}")
        return []

    battery_capacities = {
        'Batt1': batt_row['BESS1_Capacity'],
        'Batt2': batt_row['BESS2_Capacity'],
        'Batt3': batt_row['BESS3_Capacity'],
        'Batt4': batt_row['BESS4_Capacity'],
        'Batt5': batt_row['BESS5_Capacity']
    }
    print(f"Battery capacities: {battery_capacities}")

    battery_limits = [
        (
            batt,
            min_rate,
            max_rate,
            battery_capacities[batt]
        )
        for batt, (min_rate, max_rate) in adjusted_discharge_limits.items()
        if max_rate > 0 and battery_capacities[batt] > 0
    ]
    print(f"Filtered battery limits: {battery_limits}")

    best_combinations = {}
    
    # Precompute possible rates for each battery
    rate_ranges = {batt: range(int(min_rate), int(max_rate) + 1) for batt, min_rate, max_rate, _ in battery_limits}
    
    for r in range(1, len(battery_limits) + 1):
        for comb in combinations(battery_limits, r):
            rates = [rate_ranges[item[0]] for item in comb]
            for rate_comb in itertools.product(*rates):
                total_value = sum(rate_comb)
                if total_value >= totalLoad and total_value <= totalLoad + tolerance:
                    comb_with_scores = {}
                    total_score = 1
                    for i, item in enumerate(comb):
                        chosen_rate = rate_comb[i]
                        weight = chosen_rate / totalLoad
                        score = get_score_for_batt(item[0], scoreTable, rowIndex) * weight
                        comb_with_scores[item[0]] = {
                            'value': chosen_rate,
                            'score': score,
                            'min_rate': item[1],
                            'max_rate': item[2],
                        }
                        total_score *= score
                    key = tuple(sorted(comb_with_scores.keys()))
                    if key not in best_combinations or total_score > best_combinations[key][2]:
                        best_combinations[key] = (comb_with_scores, total_value, total_score)

    valid_combinations = [comb for comb in best_combinations.values() if comb[1] >= totalLoad and comb[1] <= totalLoad + tolerance]
    if valid_combinations:
        valid_combinations.sort(key=lambda x: (-x[2], abs(x[1] - totalLoad)))
        top_combinations = valid_combinations[:3]
    else:
        top_combinations = []

    return top_combinations

# top_combinations = [
#     (
#         {
#             'Batt1': {'value': 50, 'score': 0.8, 'min_rate': 10, 'max_rate': 60},
#             'Batt2': {'value': 30, 'score': 0.7, 'min_rate': 20, 'max_rate': 50}
#         },
#         80,  # total_value
#         0.56  # total_score
#     ),
#     (
#         {
#             'Batt1': {'value': 45, 'score': 0.75, 'min_rate': 10, 'max_rate': 60},
#             'Batt3': {'value': 35, 'score': 0.65, 'min_rate': 15, 'max_rate': 45}
#         },
#         80,  # total_value
#         0.4875  # total_score
#     ),
#     (
#         {
#             'Batt2': {'value': 40, 'score': 0.8, 'min_rate': 20, 'max_rate': 50},
#             'Batt3': {'value': 40, 'score': 0.7, 'min_rate': 15, 'max_rate': 45}
#         },
#         80,  # total_value
#         0.56  # total_score
#     )
# ]

In [ ]:
def print_knapsack_results(scored_combinations):
    print("Knapsack Results:\n")
    if not scored_combinations:
        print("No valid combinations found.")
        return None
    
    for i, (combo, total_value, total_score) in enumerate(scored_combinations):
        print(f"Combination {i + 1}:")
        print(f"Time Now: {datetime.now()}")
        print(f"  Total Load: {total_value} kWh")
        print(f"  Total Score: {round(total_score, 2)}")
        print(f"  Combination Details:")
        for batt, details in combo.items():
            print(f"    {batt}:")
            print(f"      Discharge Rate Chosen: {details['value']} kW/h")
            print(f"      Discharge Rates: {details['min_rate']} - {details['max_rate']} kW/h")
            print(f"      Score: {round(details['score'], 2)}")
        print("--------------------------------------------------------------\n")
    return scored_combinations[0]



# Knapsack Results:
# Combination 1:
#    Total Capacity: 1050.0
#    Total Score: 0.56
#    Batt1:
#          Discharge expected: 720.0
#          Discharge Rate: 480.0
#          Score: 0.7
#    Batt3:
#          Discharge expected: 330.0
#          Discharge Rate: 220.0
#          Score: 0.8
#    Total Discharge for this Combination: 1050.0
#    Total Score: 0.56

In [ ]:
def best_combination(scored_combinations, highlighted_sites=None):
    if not scored_combinations:
        print("Best combination based on the total score: ")
        print("There's no combination")
        return None

    if highlighted_sites is None:
        highlighted_sites = []

    best_combo = None
    highest_score = -1

    for combo, total_value, total_score in scored_combinations:
        if highlighted_sites and not any(batt in highlighted_sites for batt in combo):
            continue
        if total_score > highest_score:
            best_combo = (combo, total_value, total_score)
            highest_score = total_score

    if best_combo:
        combo, total_value, total_score = best_combo
        print("==============================================================")
        print("Best combination based on the total score: ")   
        print(f"Time Now: {datetime.now()}")
        print(f"  Total Load: {total_value} kWh")
        print(f"  Total Score: {round(total_score, 2)}")
        print(f"  Combination Details:")
        for batt, details in combo.items():
            print(f"    {batt}:")
            print(f"      Discharge Rate Chosen: {details['value']} kW/h")
            print(f"      Discharge Rates: {details['min_rate']} - {details['max_rate']} kW/h")
            print(f"      Score: {round(details['score'], 2)}")
        print("==============================================================\n")
        return combo, total_value, total_score
    else:
        print("No valid combination found.")
        return None


# ==============================================================
# Best combination based on the total score:
# Combination:
#   Total Capacity: 1050.0
#   Total Score: 0.56
#   Combination Details:
#   Batt1:
#     Discharge expected: 720.0
#     Discharge Rate: 480.0
#     Score: 0.7
#   Batt3:
#     Discharge expected: 330.0
#     Discharge Rate: 220.0
#     Score: 0.8
#   Total Discharge for this Combination: 1050.0
#   Total Score: 0.56
# ==============================================================

In [ ]:
def convert_to_tuple(best_combo, receive_time):
    print(f"Best Combo: {best_combo}")
    receive_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    result = {
        'ReceiveTime': receive_time,
        'Site1_CnD': 0,
        'Site1_Amt': 0,
        'Site2_CnD': 0,
        'Site2_Amt': 0,
        'Site3_CnD': 0,
        'Site3_Amt': 0,
        'Site4_CnD': 0,
        'Site4_Amt': 0,
        'Site5_CnD': 0,
        'Site5_Amt': 0,
    }
    
    batt_to_site = {
        'Batt1': 'Site1',
        'Batt2': 'Site2',
        'Batt3': 'Site3',
        'Batt4': 'Site4',
        'Batt5': 'Site5'
    }
    
    for batt, site in batt_to_site.items():
        if batt in best_combo:
            print(f"Setting values for {batt} -> {site}")
            result[f'{site}_CnD'] = -1
            result[f'{site}_Amt'] = best_combo[batt]['value']  # Changed to use 'value' instead of 'adjusted_total_rate'
    
    return result



# {
#     'ReceiveTime': '2023-01-01 12:00:00',
#     'Site1_CnD': -1,
#     'Site1_Amt': 480.0,  # 720.0 / 1.5
#     'Site2_CnD': 0,
#     'Site2_Amt': 0,
#     'Site3_CnD': -1,
#     'Site3_Amt': 220.0,  # 330.0 / 1.5
#     'Site4_CnD': 0,
#     'Site4_Amt': 0,
#     'Site5_CnD': 0,
#     'Site5_Amt': 0
# }

# Filtering Data

In [ ]:
# Function to round time to HH:MM:00
def round_to_minute(dt):
    return dt.replace(second=0, microsecond=0)

In [ ]:
def sum_sites_at_target_time(siteChosen):

    print("target currently: ", siteChosen)
    
    # Sum the columns Site1 to Site5
    site_columns = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5']

    # Ensure siteChosen is a DataFrame to use axis=1
    if isinstance(siteChosen, pd.Series):
        siteChosen = siteChosen.to_frame().T
        
    total_sum = siteChosen[site_columns].sum(axis=1).values[-1]
    
    return total_sum

In [ ]:
def sum_sites_before_target_time(siteChosen):
    print("target before startTime: ", siteChosen)

    # Sum the columns Site1 to Site5
    site_columns = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5']

    # Ensure siteChosen is a DataFrame to use axis=1
    if isinstance(siteChosen, pd.Series):
        siteChosen = siteChosen.to_frame().T

    total_sum = siteChosen[site_columns].sum(axis=1).values[-1]

    return total_sum

# Initialising Data

In [ ]:
last_checked_time = round_to_minute(datetime.now())
last_processed_sites_time = None


start_time_arrived = False
previous_act_not_time = None
seen_combinations = set()  # To keep track of unique combinations

last_checked_combination_time = None
best_combo = None
best_combo_start_time = None

# Initialize total discharge
total_discharge = 0.0
sitesNowDischarge_start = 0.0
sitesNowDischarge_end = 0.0

highlighted_sites = set()
immediate_change = False

timer = False

startTime_Batt = None

# Initial time setup
current_time = round_to_minute(datetime.now())
processed_minutes = set()

# Initialize dfCurrentSites DataFrame
dfCurrentSites = pd.DataFrame(columns=['Date', 'Site1', 'Site2', 'Site3', 'Site4', 'Site5'])

# While Loop

In [ ]:
def main():
    seen_combinations = set()  # To keep track of unique combinations

    last_checked_combination_time = None
    best_combo = None
    best_combo_start_time = None

    # Initialize total discharge
    total_discharge = 0.0
    sitesNowDischarge_start = 0.0
    sitesNowDischarge_end = 0.0

    highlighted_sites = set()

    timer = False
    initial_capacities = None
    current_site_data = None
    waiting_for_activation = False

    dfCurrentSites = pd.DataFrame(columns=['Date', 'Site1', 'Site2', 'Site3', 'Site4', 'Site5'])
    dfCurrentSites['Date'] = pd.to_datetime(dfCurrentSites['Date'])

    dfCurrentBatt = pd.DataFrame(columns=['ReadTime', 'BESS1_Capacity', 'BESS2_Capacity', 'BESS3_Capacity', 'BESS4_Capacity', 'BESS5_Capacity'])
    dfCurrentBatt['ReadTime'] = pd.to_datetime(dfCurrentBatt['ReadTime'])

    end_counter = 0

    # Connect to MySQL database once
    connection = reconnect()
    if connection is None:
        return  # Exit if connection cannot be established

    while True:
        done = False  # Control variable to break the loop when done
        # connection = reconnect()
        while not done:
            if not connection.is_connected():
                print("Lost database connection. Reconnecting...")
                connection = reconnect()
                if connection is None:
                    time.sleep(10)  # Wait before retrying
                    continue

            # Get data from MySQL
            try:
                dfSites = get_dfSites_from_mysql(connection)
                dfActNot = get_dfActNot_from_mysql(connection)
                dfBatt = get_dfBatt_from_mysql(connection)
            except mysql.connector.Error as e:
                print(f"Error fetching data: {e}")
                connection = reconnect()
                if connection is None:
                    time.sleep(10)  # Wait before retrying
                    continue

            if waiting_for_activation:
                # Get data from MySQL
                dfSites = get_dfSites_from_mysql(connection)
                dfActNot = get_dfActNot_from_mysql(connection)
                dfBatt = get_dfBatt_from_mysql(connection)

                # Check if new activation notice is present
                if not dfActNot.empty:
                    targetStart = dfActNot['StartTime'].iloc[-1]
                    actNotTime = dfActNot['ReceivedTime'].iloc[-1]
                    totalLoad = dfActNot['TotalLoad'].iloc[-1]
                    targetEnd = dfActNot['EndTime'].iloc[-1]
                    siteData = dfSites.iloc[-1]
                    
                    if round_to_minute(datetime.now()) >= targetStart - timedelta(minutes=10):
                        waiting_for_activation = False  # Exit the waiting state

            # Convert datetime columns and round to the nearest minute (HH:MM:00)
            print(dfBatt)
            print(dfSites)
            dfActNot['ReceivedTime'] = pd.to_datetime(dfActNot['ReceivedTime']).apply(round_to_minute)
            dfActNot['StartTime'] = pd.to_datetime(dfActNot['StartTime']).apply(round_to_minute)
            dfActNot['EndTime'] = pd.to_datetime(dfActNot['EndTime']).apply(round_to_minute)
            dfBatt['ReadTime'] = pd.to_datetime(dfBatt['ReadTime']).apply(round_to_minute)
            dfSites['Date'] = pd.to_datetime(dfSites['Date']).apply(round_to_minute)

            # Extract target times and capacity
            targetStart = dfActNot['StartTime'].iloc[-1]
            actNotTime = dfActNot['ReceivedTime'].iloc[-1]
            actNotTimeStart = dfActNot['StartTime'].iloc[-1]
            totalLoad = dfActNot['TotalLoad'].iloc[-1]
            targetEnd = dfActNot['EndTime'].iloc[-1]
            siteData = dfSites.iloc[-1]

            # Define the time window
            window_start_time = targetStart - timedelta(minutes=10)
            receivedTime = round_to_minute(datetime.now())  # HH:MM:00
            # Checking time difference
            time_diff = targetStart - receivedTime  
            
            # Initialize discharge counters
            discharge_counters = {f'Batt{i}_discharge': 0 for i in range(1, 6)}

            # Calculate targetSpan and targetSpan_hours
            targetSpan = targetEnd - targetStart
            
            # Define quarter intervals of targetSpan
            quarter_interval = targetSpan / 4
            ###################################################################
            # Before Scenario 1: Checking for 10 min window
            if receivedTime < window_start_time:
                print(f"Checking for the 10-minute window: {time_diff}")
            ###################################################################
            # Scenario 1: Determine best combination within 10 minutes before targetStart
            elif window_start_time <= round_to_minute(datetime.now()) < targetStart:
                # Get data from MySQL
                connection = reconnect()
                receivedTime = round_to_minute(datetime.now())
                dfSites = get_dfSites_from_mysql(connection)
                dfActNot = get_dfActNot_from_mysql(connection)
                dfBatt = get_dfBatt_from_mysql(connection)
                print(f"Minutes before activation time: {time_diff}")
                siteData = get_new_site_row(connection)
                battData = get_new_batt_row(connection)
                # Ensure siteData is a dictionary of floats
                siteData_dict = {f'Site{i+1}': float(siteData[f'Site{i+1}']) for i in range(5)}

                knapsack_result = fractional_knapsack(totalLoad, siteData_dict, scoreTable, rowIndex=-1, connection=connection, target_time=receivedTime, tolerance=10.0)
                scored_combinations = calculate_combination_scores(knapsack_result, totalLoad)
                print_knapsack_results(scored_combinations)
                best_combo = best_combination(scored_combinations)
                if not best_combo:
                    print("No valid combinations found in Scenario 1.")
                done = True  # Reset the inner loop
            ###################################################################
            # Scenario 2: Determine best combination at targetStart
            elif round_to_minute(datetime.now()) == targetStart:
                # Get data from MySQL
                connection = reconnect()
                receivedTime = round_to_minute(datetime.now())
                dfSites = get_dfSites_from_mysql(connection)
                dfActNot = get_dfActNot_from_mysql(connection)
                dfBatt = get_dfBatt_from_mysql(connection)
                print(f"Time Now: {datetime.now()}")
                # Store initial capacities at targetStart
                initial_batt_capacity = get_specific_batt_row(connection, targetStart)
                initial_capacities = {
                    'ReadTime': initial_batt_capacity['ReadTime'],
                    'BESS1_Capacity': initial_batt_capacity['BESS1_Capacity'],
                    'BESS2_Capacity': initial_batt_capacity['BESS2_Capacity'],
                    'BESS3_Capacity': initial_batt_capacity['BESS3_Capacity'],
                    'BESS4_Capacity': initial_batt_capacity['BESS4_Capacity'],
                    'BESS5_Capacity': initial_batt_capacity['BESS5_Capacity']
                }

                print("Initial capacities at targetStart:", initial_capacities)
                siteData = get_new_site_row(connection)
                battData = get_new_batt_row(connection)
                timer = True
                # Ensure siteData is a dictionary of floats
                siteData_dict = {f'Site{i+1}': float(siteData[f'Site{i+1}']) for i in range(5)}
                knapsack_result = fractional_knapsack(totalLoad, siteData_dict, scoreTable, rowIndex=-1, connection=connection, target_time=receivedTime, tolerance=10.0)
                scored_combinations = calculate_combination_scores(knapsack_result, totalLoad)
                best_combo = best_combination(scored_combinations)
                
                if best_combo:
                    best_combo_dict, total_value, total_score = best_combo  # Unpack the best_combo
                    best_combo_start_time = receivedTime
                    # Dispense battery combination (best_combination)
                    if best_combo_dict:
                        receive_time = siteData['Date']
                        best_combo_tuple = convert_to_tuple(best_combo_dict, receive_time)
                        print(best_combo_tuple)
                        print(f"Dispense tuple: {best_combo_tuple}")
                        execute_query(connection, insert_query, best_combo_tuple)
                    else:
                        print("No valid combinations found at targetStart.")
                        waiting_for_activation = True  # Set flag to wait for a new activation
                else:
                    print("No best combination found at targetStart.")
                    waiting_for_activation = True  # Set flag to wait for a new activation
                done = True  # Reset the inner loop
            ###################################################################
            # Scenario 3: Handling past targetStart
            elif targetStart + timedelta(minutes=1) <= round_to_minute(datetime.now()) < targetEnd:
                # Get data from MySQL
                connection = reconnect()
                receivedTime = round_to_minute(datetime.now())
                dfSites = get_dfSites_from_mysql(connection)
                dfActNot = get_dfActNot_from_mysql(connection)
                dfBatt = get_dfBatt_from_mysql(connection)
                siteData = get_new_site_row(connection)
                battData = get_new_batt_row(connection)
                print(f"siteData: {siteData}")
                print(f"battData: {battData}")

                # Append current site data to dfCurrentSites
                new_row_sites = pd.DataFrame({
                    'Date': [siteData['Date']],
                    'Site1': [siteData['Site1']],
                    'Site2': [siteData['Site2']],
                    'Site3': [siteData['Site3']],
                    'Site4': [siteData['Site4']],
                    'Site5': [siteData['Site5']]
                })
                if dfCurrentSites.empty or not dfCurrentSites.iloc[-1].equals(new_row_sites.iloc[0]):
                    dfCurrentSites = pd.concat([dfCurrentSites, new_row_sites], ignore_index=True)
                print("dfCurrentSites", dfCurrentSites)

                new_row_batt = pd.DataFrame({
                    'ReadTime': [battData['ReadTime']],
                    'BESS1_Capacity': [battData['BESS1_Capacity']],
                    'BESS2_Capacity': [battData['BESS2_Capacity']],
                    'BESS3_Capacity': [battData['BESS3_Capacity']],
                    'BESS4_Capacity': [battData['BESS4_Capacity']],
                    'BESS5_Capacity': [battData['BESS5_Capacity']]
                })
                if dfCurrentBatt.empty or not dfCurrentBatt.iloc[-1].equals(new_row_batt.iloc[0]):
                    dfCurrentBatt = pd.concat([dfCurrentBatt, new_row_batt], ignore_index=True)
                print("dfCurrentBatt", dfCurrentBatt)

                new_best_combo = None  # Initialize new_best_combo

                if not dfCurrentSites.empty:
                    # Pick the last record
                    data_now = dfCurrentSites.iloc[-1]
                    print(f"siteData before filtering: {data_now}")

                    # Ensure that the values are properly extracted
                    data_now_dict = {col: float(data_now[col]) for col in data_now.index if col.startswith('Site')}
                    valid_sites = {key: value for key, value in data_now_dict.items() if value >= 0}

                    print(f"valid_sites after filtering: {valid_sites}")
                    if not valid_sites:
                        print("All sites have values < 0. Skipping knapsack calculation.")
                        knapsack_result = []
                    else:
                        highlighted_sites = list(valid_sites.keys())
                        # Ensure siteData is a dictionary of floats
                        siteData_dict = {f'Site{i+1}': float(siteData[f'Site{i+1}']) for i in range(5)}
                        knapsack_result = fractional_knapsack(totalLoad, siteData_dict, scoreTable, rowIndex=-1, connection=connection, target_time=receivedTime, tolerance=10.0)

                        if knapsack_result:
                            scored_combinations = calculate_combination_scores(knapsack_result, totalLoad)
                            new_best_combo = best_combination(scored_combinations)

                        if new_best_combo:
                            best_combo_dict, total_value, total_score = new_best_combo
                            receive_time = siteData['Date']
                            best_combo_tuple = convert_to_tuple(best_combo_dict, receive_time)
                            print(f"Best combination: {best_combo_tuple}")
                            # execute_query(connection, insert_query, best_combo_tuple)

                # Check every quarter of targetSpan and send the best combination to MySQL
                if last_checked_combination_time is None or (round_to_minute(datetime.now()) - last_checked_combination_time) == quarter_interval:
                    last_checked_combination_time = datetime.now()
                    print(datetime.now().strftime('%H:%M:%S'))

                    # Execute query for the best combination found in the last quarter interval
                    if new_best_combo:
                        best_combo_dict, total_value, total_score = new_best_combo
                        receive_time = siteData['Date']
                        best_combo_tuple = convert_to_tuple(best_combo_dict, receive_time)
                        print(f"Best combination after every quarter interval: {best_combo_tuple}")
                        execute_query(connection, insert_query, best_combo_tuple)

                    elif not new_best_combo:
                        print("No new valid combinations found.")

                    # Calculate discharge for each battery
                    for batt in best_combo_dict:
                        discharge_counters[f'{batt}_discharge'] += best_combo_dict[batt]['value']
                
                    print(f"###########################################################")
                    print(f"Discharge amounts from {targetStart} to {round_to_minute(datetime.now())}:")
                    targetStart_site_data = get_specific_site_row(connection, targetStart)
                    current_site_data = get_specific_site_row(connection, receivedTime)
                    sitesNowDischarge_start = sum_sites_before_target_time(targetStart_site_data)
                    sitesNowDischarge_current = sum_sites_at_target_time(current_site_data)
                    total_discharge = sitesNowDischarge_start - sitesNowDischarge_current
                    print(f"Total discharge = {total_discharge} kWh (from SitesPowerNow)")
                    print(f"###########################################################\n")

                    # Calculate and print discharge for each battery from dfBatt
                    latest_row = get_specific_batt_row(connection, receivedTime)
                    discharge_counters_end = {
                        'Batt1': initial_capacities['BESS1_Capacity'] - latest_row['BESS1_Capacity'],
                        'Batt2': initial_capacities['BESS2_Capacity'] - latest_row['BESS2_Capacity'],
                        'Batt3': initial_capacities['BESS3_Capacity'] - latest_row['BESS3_Capacity'],
                        'Batt4': initial_capacities['BESS4_Capacity'] - latest_row['BESS4_Capacity'],
                        'Batt5': initial_capacities['BESS5_Capacity'] - latest_row['BESS5_Capacity']
                    }
                    print(f"------------------------------------------------------------")
                    print(f"Discharge counters at {round_to_minute(datetime.now())}:")
                    for batt, discharge in discharge_counters_end.items():
                        print(f"  {batt}: {discharge} kWh")
                    print(f"------------------------------------------------------------")
                    
        
                done = True  # Reset the inner loop
            ###################################################################
            # Scenario 4: targetEnd arrived
            elif round_to_minute(datetime.now()) == targetEnd:
                # Get data from MySQL
                connection = reconnect()
                receivedTime = round_to_minute(datetime.now())
                dfSites = get_dfSites_from_mysql(connection)
                dfActNot = get_dfActNot_from_mysql(connection)
                dfBatt = get_dfBatt_from_mysql(connection)
                print(datetime.now())
                targetEndData = get_specific_site_row(connection, targetEnd)
                
                if not targetEndData.empty:
                    if best_combo_start_time:
                        print(f"###########################################################")
                        print(f"Discharge amounts from {targetStart} to {targetEnd}:")
                        targetStart_site_data = get_specific_site_row(connection, targetStart)
                        sitesNowDischarge_start = sum_sites_before_target_time(targetStart_site_data)
                        sitesNowDischarge_end = sum_sites_at_target_time(targetEndData)
                        total_discharge = sitesNowDischarge_start - sitesNowDischarge_end
                        print(f"    Start sum: {sitesNowDischarge_start}")
                        print(f"    End sum: {sitesNowDischarge_end}")
                        print(f"Total discharge = {total_discharge} kWh (from SitesPowerNow)")
                        print(f"###########################################################\n")
                        

                    # Calculate and print discharge for each battery from dfBatt
                    latest_row = get_specific_batt_row(connection, receivedTime)

                    discharge_counters_end = {
                        'Batt1': initial_capacities['BESS1_Capacity'] - latest_row['BESS1_Capacity'],
                        'Batt2': initial_capacities['BESS2_Capacity'] - latest_row['BESS2_Capacity'],
                        'Batt3': initial_capacities['BESS3_Capacity'] - latest_row['BESS3_Capacity'],
                        'Batt4': initial_capacities['BESS4_Capacity'] - latest_row['BESS4_Capacity'],
                        'Batt5': initial_capacities['BESS5_Capacity'] - latest_row['BESS5_Capacity']
                    }

                    print(f"------------------------------------------------------------")
                    print(f"Discharge counters at {receivedTime}:")
                    for batt, discharge in discharge_counters_end.items():
                        print(f"  {batt}: {discharge} kWh")
                    print(f"------------------------------------------------------------")

                    params = {'ReceiveTime': targetEnd}
                    execute_query(connection, end_query, params)
                    end_counter += 1

                    done = True  # Reset the inner loop
                else:
                    print(f"No data found for time {targetEnd} or nearest available time.")
                    params = {'ReceiveTime': targetEnd}
                    execute_query(connection, end_query, params)
                    end_counter += 1
                    done = True  # Reset the inner loop

            ###################################################################
            # Scenario 5: If current time is after targetEnd
            elif round_to_minute(datetime.now()) > targetEnd:
                # Get data from MySQL
                if end_counter == 0:
                    params = {'ReceiveTime': targetEnd}
                    execute_query(connection, end_query, params)
                receivedTime = round_to_minute(datetime.now())
                connection = reconnect()
                dfActNot = get_dfActNot_from_mysql(connection)
                print(f"After targetEnd: {receivedTime}")
                if actNotTime <= receivedTime:  # Ensure the activation is current or in the past
                    actNotCheck = dfActNot['ReceivedTime'].iloc[-1]  # Pull the newest record's ReceivedTime
                    actNotCheckStart = dfActNot['StartTime'].iloc[-1]

                    if actNotCheck == actNotTime or actNotTimeStart == actNotCheckStart:
                        print("No new activations.")
                        timer = False
                    elif actNotCheck < actNotTime:  # Added `elif` to properly handle this case
                        newest_act_not_start_time = dfActNot['StartTime'].iloc[-1]
                        newest_act_not_time = newest_act_not_start_time - timedelta(minutes=10)
                        
                        if newest_act_not_time <= targetEnd <= newest_act_not_start_time:
                            print("New activation detected, cannot proceed with new activation since current one has not ended.")
                            continue
                        else:
                            best_combo = None  # Reset the best combination
                            seen_combinations.clear()  # Clear the set of seen combinations
                            last_checked_combination_time = None  # Reset the last checked combination time
                            total_discharge = 0.0  # Reset total discharge
                            sitesNowDischarge_start = 0.0
                            sitesNowDischarge_end = 0.0
                            timer = False
                            highlighted_sites.clear()
                            dfCurrentSites = pd.DataFrame(columns=dfCurrentSites.columns)
                            current_site_data = None
                            dfCurrentBatt = pd.DataFrame(columns=dfCurrentBatt.columns)
                            end_counter = 0
                            print("New activation detected. Resetting the state.")
                            done = True
            
            elif timer:
                current = round_to_minute(datetime.now())
                remaining_time = targetEnd - current
                minutes = divmod(remaining_time.total_seconds(), 60)
                print(f"Time remaining: {round(minutes[0])} minutes")

            time.sleep(60)

In [ ]:
if __name__ == "__main__":
    main()